In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip ./glove.6B.zip
!pip install kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!unzip ./jigsaw-toxic-comment-classification-challenge.zip
!unzip ./train.csv.zip
!unzip ./test.csv.zip
!unzip ./test_labels.csv.zip

--2022-07-06 18:54:54--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-07-06 18:54:55--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-07-06 18:54:55--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes
 85% 45.0M/52.6M [00:00<00:00, 49.7MB/s]
100% 52.6M/52.6M [00:00<00:00, 58.3MB/s]
Archive:  ./jigsaw-toxic-comment-classification-challenge.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           
Archive:  ./train.csv.zip
  inflating: train.csv               
Archive:  ./test.csv.zip
  inflating: test.csv                
Archive:  ./test_labels.csv.zip
  inflating: test_labels.csv         


In [ ]:
import sys
import os
import keras
import sklearn
import numpy as np
import pandas as pd

In [ ]:
df_train=pd.read_csv("./train.csv")

In [ ]:
target_label=df_train.iloc[:,2:].values

In [ ]:
comment_text=df_train["comment_text"].fillna("DUMMY_VALUE").values

In [ ]:
word2vec={}
f=open("./glove.6B.100d.txt")    
for line in f:
        temp=line.split()
        word=temp[0]
        vec=np.asarray(temp[1:],dtype="float32")
        word2vec[word]=vec

In [ ]:
tokenizer=keras.preprocessing.text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(comment_text)
sequance=tokenizer.texts_to_sequences(comment_text)
word2index=tokenizer.word_index

In [ ]:
max_sequance_len=max(len(s) for s in sequance)

In [ ]:
sequance=keras.preprocessing.sequence.pad_sequences(sequance,maxlen=max_sequance_len)

In [ ]:
embedding_matrix=np.zeros((len(word2index.keys()),100))

In [ ]:
for word,i in word2index.items():
    if i<len(word2index.keys()):
        embedding_vec=word2vec.get(word)
        if  embedding_vec is not None:
            embedding_matrix[i]=embedding_vec

In [ ]:
embedding_layer=keras.layers.Embedding(len(word2index.keys()),100,weights=[embedding_matrix],trainable=False
                                        ,input_length=max_sequance_len)

In [ ]:
input_=keras.layers.Input(shape=(max_sequance_len,))
x=embedding_layer(input_)
x=keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x=keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x=keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x=keras.layers.GlobalMaxPool1D()(x)
y=keras.layers.Dense(6,activation="sigmoid")(x)
model=keras.Model(input_,y)

In [ ]:
model.compile(optimizer="rmsprop",
              loss=keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"])

In [ ]:
model.fit(sequance,target_label,epochs=10,batch_size=32, validation_split=0.2)

Epoch 1/10
3990/3990 [==============================] - 766s 189ms/step - loss: 0.0887 - accuracy: 0.9929 - val_loss: 0.0569 - val_accuracy: 0.9941
Epoch 2/10
3990/3990 [==============================] - 759s 190ms/step - loss: 0.0558 - accuracy: 0.9942 - val_loss: 0.0536 - val_accuracy: 0.9941
Epoch 3/10
3990/3990 [==============================] - 733s 184ms/step - loss: 0.0511 - accuracy: 0.9942 - val_loss: 0.0504 - val_accuracy: 0.9941
Epoch 4/10
3990/3990 [==============================] - 733s 184ms/step - loss: 0.0493 - accuracy: 0.9939 - val_loss: 0.0570 - val_accuracy: 0.9903
Epoch 5/10
3990/3990 [==============================] - 731s 183ms/step - loss: 0.0481 - accuracy: 0.9938 - val_loss: 0.0488 - val_accuracy: 0.9941
Epoch 6/10
3990/3990 [==============================] - 749s 188ms/step - loss: 0.0471 - accuracy: 0.9937 - val_loss: 0.0494 - val_accuracy: 0.9941
Epoch 7/10
3990/3990 [==============================] - 735s 184ms/step - loss: 0.0457 - accuracy: 0.9936 - val_

In [ ]:
df_test=pd.read_csv("./test.csv")

In [ ]:
target_label_test=pd.read_csv("./test_labels.csv")

In [ ]:
comment_text_test=df_test["comment_text"].fillna("DUMMY_VALUE").values

In [ ]:
sequance_test=tokenizer.texts_to_sequences(comment_text_test)

In [ ]:
sequance_test=keras.preprocessing.sequence.pad_sequences(sequance_test,maxlen=max_sequance_len)

In [ ]:
model.evaluate(sequance_test,target_label_test.iloc[:,1:].values)

4787/4787 [==============================] - 335s 70ms/step - loss: -4.1238 - accuracy: 0.9966


[-4.123835563659668, 0.9965592622756958]

In [ ]:
sequance_try=tokenizer.texts_to_sequences(["fuck you"])
sequance_try=keras.preprocessing.sequence.pad_sequences(sequance_try,maxlen=max_sequance_len)

In [ ]:
model.predict(sequance_try)

array([[0.99885   , 0.25203386, 0.994696  , 0.00976737, 0.8180527 ,
        0.01936322]], dtype=float32)

In [ ]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
